In [1]:
import pandas as pd #import pandas library
import numpy as np  #import numoy library
import matplotlib.pyplot as plt  #import pyplot library
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('CKD.csv') #taking excel data to pandas dataframe
df.head(3) #Show first three rows of the dataframe

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes


In [3]:
df.shape # get the shape of the dataframe

(399, 25)

In [4]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [5]:
df['classification'].value_counts() #get number of types in output

classification
yes    249
no     150
Name: count, dtype: int64

In [6]:
df['pc'].unique()

array(['abnormal', 'normal'], dtype=object)

In [7]:
df.describe #df description

<bound method NDFrame.describe of            age         bp sg   al   su     rbc        pc         pcc  \
0     2.000000  76.459948  c  3.0  0.0  normal  abnormal  notpresent   
1     3.000000  76.459948  c  2.0  0.0  normal    normal  notpresent   
2     4.000000  76.459948  a  1.0  0.0  normal    normal  notpresent   
3     5.000000  76.459948  d  1.0  0.0  normal    normal  notpresent   
4     5.000000  50.000000  c  0.0  0.0  normal    normal  notpresent   
..         ...        ... ..  ...  ...     ...       ...         ...   
394  51.492308  70.000000  a  0.0  0.0  normal    normal  notpresent   
395  51.492308  70.000000  c  0.0  2.0  normal    normal  notpresent   
396  51.492308  70.000000  c  3.0  0.0  normal    normal  notpresent   
397  51.492308  90.000000  a  0.0  0.0  normal    normal  notpresent   
398  51.492308  80.000000  a  0.0  0.0  normal    normal  notpresent   

             ba         bgr  ...        pcv            wc        rc  htn   dm  \
0    notpresent  148

In [8]:
df1=pd.get_dummies(df,drop_first=True).astype(float)
df1.head(3)

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
df1.shape #get df shape 

(399, 28)

In [10]:
df1.columns #get df columns name 

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [11]:
indep=df1.drop('classification_yes',axis=1) #input data
dep=df1['classification_yes'] #outout data

In [12]:
from sklearn.model_selection import train_test_split #library to split training data and split data
X_train,X_test,y_train,y_test = train_test_split(indep,dep,test_size=0.3,random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler #Stndadizing the dataframe data
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier #importing Random Forest Classifier
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {'criterion':['gini','entropy','log_loss'],  #parameters are given in json
              'max_features': ['sqrt','log2'],
              'max_depth': [None, 10, 20, 30],
              'n_estimators':[10,100,200, 300],
         'bootstrap': [True, False]   
         } 



grid = GridSearchCV(RandomForestClassifier(random_state=42), params, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') #model is created
grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)

print("\n")
from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_)
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[[49  1]
 [ 2 68]]


              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97        50
         1.0       0.99      0.97      0.98        70

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120



{'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100}


Weighted F1-score: 0.9750338343186836


In [15]:
from sklearn.tree import DecisionTreeClassifier #importing Decision Tree Classifier
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
    'criterion': ['gini', 'entropy', 'log_loss'],  #parameters are given in json
    'splitter': ['best', 'random'],                 
    'max_features': ['sqrt', 'log2', None],         
    'max_depth': [None, 5, 10, 20, 30, 50],          
    'class_weight': [None, 'balanced']               
}
 



grid = GridSearchCV(DecisionTreeClassifier(random_state=42), params, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') #model is created
grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_)
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[[49  1]
 [ 5 65]]


              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94        50
         1.0       0.98      0.93      0.96        70

    accuracy                           0.95       120
   macro avg       0.95      0.95      0.95       120
weighted avg       0.95      0.95      0.95       120



{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'splitter': 'random'}


Weighted F1-score: 0.9502262443438915


In [16]:
from sklearn.neighbors import KNeighborsClassifier #importing K Nearest Neighbors Classifier
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
    'n_neighbors': [3,5,7,9,11],    #parameters are given in json
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],               
    'metric': ['minkowski','euclidean','manhattan'],         
    'n_jobs': [-1, 5, 10, 20, 30, 50]                      
}
 



grid = GridSearchCV(KNeighborsClassifier(),params, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') #model is created
grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_)
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


C:\Users\abin.k\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[[39 11]
 [22 48]]


              precision    recall  f1-score   support

         0.0       0.64      0.78      0.70        50
         1.0       0.81      0.69      0.74        70

    accuracy                           0.72       120
   macro avg       0.73      0.73      0.72       120
weighted avg       0.74      0.72      0.73       120



{'algorithm': 'auto', 'metric': 'manhattan', 'n_jobs': -1, 'n_neighbors': 7}


Weighted F1-score: 0.7269013199245757


In [17]:
from sklearn.svm import SVC #importing Support Vector Machine
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
    'C': [0.1, 1, 10, 100],               # Regularization strength
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel type
    'gamma': ['scale', 'auto'] # Kernel coefficient
}

 
grid = GridSearchCV(SVC(random_state=42), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[[49  1]
 [ 2 68]]


              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97        50
         1.0       0.99      0.97      0.98        70

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120



{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


Weighted F1-score: 0.9750338343186836


In [18]:
from sklearn.linear_model import LogisticRegression #importing Logistic Regression
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
    'solver':['newton-cg', 'lbfgs','newton-cholesky', 'liblinear','sag', 'saga'],
             'penalty':['l2']
}

 
grid = GridSearchCV(LogisticRegression(random_state=42), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[[49  1]
 [ 2 68]]


              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97        50
         1.0       0.99      0.97      0.98        70

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120



{'penalty': 'l2', 'solver': 'liblinear'}


Weighted F1-score: 0.9750338343186836


In [19]:
from sklearn.naive_bayes import GaussianNB #importing Gaussian Naive Baye's 
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

# params =  {
#     'var_smoothing': np.logspace(0, -9, num=100)
# }
params = {
  'var_smoothing': np.logspace(0, -9, num=100)
}
 
grid = GridSearchCV(GaussianNB(), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[[49  1]
 [ 7 63]]


              precision    recall  f1-score   support

         0.0       0.88      0.98      0.92        50
         1.0       0.98      0.90      0.94        70

    accuracy                           0.93       120
   macro avg       0.93      0.94      0.93       120
weighted avg       0.94      0.93      0.93       120



{'var_smoothing': 2.848035868435805e-09}


Weighted F1-score: 0.9337275884727307


In [20]:
from sklearn.naive_bayes import MultinomialNB #importing Multinomial Naive Baye's 
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
   "alpha": np.linspace(0.1, 1.0, 10),
            "fit_prior": [True, False]
}
 
grid = GridSearchCV(MultinomialNB(), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[[50  0]
 [15 55]]


              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87        50
         1.0       1.00      0.79      0.88        70

    accuracy                           0.88       120
   macro avg       0.88      0.89      0.87       120
weighted avg       0.90      0.88      0.88       120



{'alpha': 0.1, 'fit_prior': True}


Weighted F1-score: 0.8756521739130434


In [21]:
from sklearn.naive_bayes import BernoulliNB #importing Bernoulli Naive Baye's 
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
              "alpha": np.linspace(0.1, 1.0, 10),
            "binarize": [0.0, 0.5, 1.0],
            "fit_prior": [True, False]
}
 
grid = GridSearchCV(BernoulliNB(), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[[49  1]
 [ 3 67]]


              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96        50
         1.0       0.99      0.96      0.97        70

    accuracy                           0.97       120
   macro avg       0.96      0.97      0.97       120
weighted avg       0.97      0.97      0.97       120



{'alpha': 0.1, 'binarize': 0.0, 'fit_prior': True}


Weighted F1-score: 0.9667519181585678


In [22]:
from sklearn.naive_bayes import ComplementNB #importing Complement Naive Baye's 
from sklearn.model_selection import GridSearchCV #importing Grid Search Cross Validation

params = {
            "alpha": np.linspace(0.1, 1.0, 10),
            "norm": [True, False]
}
 
grid = GridSearchCV(ComplementNB(), params, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix #confusion matrix is imported
CM = confusion_matrix(y_test,y_pred)
print(CM)
print("\n")

from sklearn.metrics import classification_report #classification report is imported
clf_report = classification_report(y_test,y_pred,zero_division=0)
print(clf_report)
print("\n")
print(grid.best_params_) 
print("\n")
f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1-score:", f1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[[50  0]
 [15 55]]


              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87        50
         1.0       1.00      0.79      0.88        70

    accuracy                           0.88       120
   macro avg       0.88      0.89      0.87       120
weighted avg       0.90      0.88      0.88       120



{'alpha': 0.1, 'norm': False}


Weighted F1-score: 0.8756521739130434
